In [1]:
import numpy as np
import pandas as pd
import os
import itertools

In [2]:
keypoints_path = "/content/drive/MyDrive/Exercise Tracking/keypoints"
features_path = "/content/drive/MyDrive/Exercise Tracking/features"

os.makedirs(features_path, exist_ok=True)

In [3]:
# Function to calculate Euclidean distance between two points
def calculate_distance(point1, point2):
    return np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2 + (point1[2] - point2[2])**2)

# Function to calculate angle between three points
def calculate_angle(point1, point2, point3):
    vector1 = np.array(point1) - np.array(point2)
    vector2 = np.array(point3) - np.array(point2)
    cosine_angle = np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))
    cosine_angle = np.clip(cosine_angle, -1.0, 1.0)
    angle = np.degrees(np.arccos(cosine_angle))
    return angle

In [4]:
# Function to normalize features
def normalize_features(features_df, normalization_factors):
    normalized_df = features_df.copy()
    for col in features_df.columns:
        if col.startswith("distance") or col.startswith("angle"):
            normalized_df[col] = features_df[col] / normalization_factors[col]
    return normalized_df

In [5]:
# Feature engineering function
def engineer_features(keypoints_csv, output_csv):
    keypoints_df = pd.read_csv(keypoints_csv)
    features = []

    for _, row in keypoints_df.iterrows():
        frame = row['frame']
        keypoints = np.array(row[1:]).reshape(-1, 3)

        # Initialize features for the current frame
        feature_row = {"frame": frame}

        # Compute all pairwise distances
        for i, j in itertools.combinations(range(len(keypoints)), 2):
            point1 = keypoints[i]
            point2 = keypoints[j]
            if None not in point1 and None not in point2:
                distance = calculate_distance(point1, point2)
            else:
                distance = None
            feature_row[f"distance_{i}_{j}"] = distance

        # Compute all meaningful angles (center point with two neighbors)
        for i, j, k in itertools.combinations(range(len(keypoints)), 3):
            point1 = keypoints[i]
            point2 = keypoints[j]  # Central point
            point3 = keypoints[k]
            if None not in point1 and None not in point2 and None not in point3:
                angle = calculate_angle(point1, point2, point3)
            else:
                angle = None
            feature_row[f"angle_{i}_{j}_{k}"] = angle

        # Add feature row to the list
        features.append(feature_row)

    # Convert features to a DataFrame
    features_df = pd.DataFrame(features)

    normalization_factors = {
        col: features_df[col].max() if col.startswith("distance") else 180
        for col in features_df.columns if col.startswith("distance") or col.startswith("angle")
    }

    # Normalize features
    normalized_features_df = normalize_features(features_df, normalization_factors)

    # Save normalized features to CSV
    normalized_features_df.to_csv(output_csv, index=False)
    print(f"Normalized features saved to {output_csv}")

In [6]:
def process_all_features():
    for file in os.listdir(keypoints_path):
        if file.endswith("_keypoints.csv"):
            exercise_name = file.replace("_keypoints.csv", "")
            keypoints_csv = os.path.join(keypoints_path, file)
            output_csv = os.path.join(features_path, f"{exercise_name}_features.csv")
            engineer_features(keypoints_csv, output_csv)

In [7]:
process_all_features()

Normalized features saved to /content/drive/MyDrive/Exercise Tracking/features/Seated_Neck_Flexion_7_features.csv
Normalized features saved to /content/drive/MyDrive/Exercise Tracking/features/Seated_Neck_Flexion_8_features.csv
Normalized features saved to /content/drive/MyDrive/Exercise Tracking/features/Seated_Neck_Flexion_1_features.csv
Normalized features saved to /content/drive/MyDrive/Exercise Tracking/features/Seated_Neck_Flexion_2_features.csv
Normalized features saved to /content/drive/MyDrive/Exercise Tracking/features/Seated_Neck_Flexion_3_features.csv
Normalized features saved to /content/drive/MyDrive/Exercise Tracking/features/Seated_Neck_Flexion_4_features.csv
Normalized features saved to /content/drive/MyDrive/Exercise Tracking/features/Seated_Neck_Flexion_5_features.csv
Normalized features saved to /content/drive/MyDrive/Exercise Tracking/features/Seated_Neck_Flexion_6_features.csv
Normalized features saved to /content/drive/MyDrive/Exercise Tracking/features/Seated_Ne